## Задача
Создайте модель, обрабатывающую фрагмент текста и определяющую
какой вид продукции в нём содержится.

Выполненное задание пришлите ссылкой на Google Collab.
Датасет поделите на 80% / 20% - обучающая/тестовая выборки.

Используйте библиотеку PyTorch

Датасет: https://axe.inline-ltd.ru/data/meatinfo.csv

Виды продукции (брать только виды продукции, для которых в датасете есть не менее 500 примеров):

Баранина
Ягнятина
Индейка
Говядина
Свинина
Кура
Цыплено
Гусь
Буйволятина
Оленина
Конина
Телятина
Кролик
Утка
Куропатка
Перепел
Глухарь
Страус
Заяц
Кенгуру
Изюбр
Кабан
Коза
Косуля
Лось
Марал
Медвежатина
Бобер
Цесарка
Нутрия
Рябчик
Тетерев
Фазан
Як


Примеры входных текстов и видов продукции:
Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг. -> Свинина

Мясо премиум Предлагаем котлетное мясо мраморной говядины. -> Говядина

спинка цб -> Цыпленок

Проверьте вашу модель на образцах
Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)
Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.

Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:

Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.

```py
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_lg
```

In [ ]:
import pandas as pd
import torch
import re
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from pymystem3 import Mystem
from torchtext import data
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch import optim
from tqdm.notebook import tqdm
import numpy as np
import scipy
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [ ]:
df = pd.read_csv('https://axe.inline-ltd.ru/data/meatinfo.csv', sep = ';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17893 entries, 0 to 17892
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17893 non-null  object
 1   mtype   17892 non-null  object
dtypes: object(2)
memory usage: 279.7+ KB


In [ ]:
df['mtype'].value_counts()

Говядина                                                                                   8422
Свинина                                                                                    3050
Кура                                                                                       1571
Индейка                                                                                    1337
Баранина                                                                                   1116
Цыпленок                                                                                    942
Кролик                                                                                      334
Утка                                                                                        195
Оленина                                                                                     193
Конина                                                                                      176
Гусь                                    

In [ ]:
df['text'].iloc[100]

'Антрекот (филейный край) охл. Москва.\nАнтрекот (филейный край) говяжий охлажденный .'

#### Преобразуем все виды продукции нижний регистр и сделаем замену неправльно написанных элементов в столбце mtype на необходимые категории

In [ ]:
df['mtype'] = df['mtype'].str.lower()

df.loc[df['mtype'] == ' лопаточная часть (chuck) буйвол ', 'mtype'] = 'буйволятина'
df.loc[df['mtype'] == 'говядина, полутуши, 1 категория,  охл., россия, подвес, в наличии, 10 тонн, 270 руб. кг', 'mtype'] = 'говядина'
df.loc[df['mtype'] == 'свиниеа', 'mtype'] = 'свинина'

df['mtype'].value_counts()

говядина                       8424
свинина                        3054
кура                           1571
индейка                        1339
баранина                       1116
цыпленок                        944
кролик                          334
утка                            196
оленина                         193
конина                          176
гусь                            125
телятина                         98
ягнятина                         76
буйволятина                      76
перепел                          54
кабан                            24
лось                             20
страус                           10
косуля                            8
куропатка                         7
цесарка                           7
марал                             7
фазан                             4
медвежатина                       4
изюбр                             3
feb-20                            2
нутрия                            2
заяц                        

#### Необходимо взять только виды продукции, которые имеют не менее 500 примеров, также необходимо сделать кодирование признаков

In [ ]:
df = df.loc[df['mtype'].isin(list(df['mtype'].value_counts().loc[lambda x : x >= 500].index))]
df['mtype'].value_counts()

говядина    8424
свинина     3054
кура        1571
индейка     1339
баранина    1116
цыпленок     944
Name: mtype, dtype: int64

In [ ]:
display(df.head())

le = LabelEncoder().fit(df["mtype"])
df['mtype'] = le.transform(df['mtype'])

display(df.head())

,text,mtype
0,12 частей баранина 12 частей баранина,баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",баранина
3,"Баранина, 12 частей, охл.",баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",баранина


,text,mtype
0,12 частей баранина 12 частей баранина,0
1,"Баранина, 12 частей, зам. цена 260 руб.",0
2,"Баранина, 12 частей, зам. цена 315 руб.",0
3,"Баранина, 12 частей, охл.",0
4,"Баранина, 12 частей, охл. цена 220 руб.",0


## Важно
Данные имеют очень высокий дисбаланс классов. Для бейзлайн решения бороться с дисбалансом не будем

In [ ]:
df['mtype'].value_counts()

1    8424
4    3054
3    1571
2    1339
0    1116
5     944
Name: mtype, dtype: int64

#### Необходимо оставить только буквенные записи для каждого текста, также проведём лемантизацию текстов

In [ ]:
df['text'].iloc[101]

'Антрекот гов Россия ГОСТ Говядина, антрекот, бк, охл., Россия, коробки, склад, от 100 кг до 500 кг'

In [ ]:
m = Mystem()
df['text'] = df['text'].apply(lambda x : ''.join(m.lemmatize(x)))
df['text'].iloc[101]

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


'антрекот гов россия гост говядина, антрекот, бк, охла., россия, коробка, склад, от 100 кг до 500 кг\n'

In [ ]:
df['text'] = df['text'].apply(lambda x : ' '.join(re.sub(r'[^а-яА-ЯёЁ]', ' ', x.lower()).split()))
df['text'].iloc[101]

'антрекот гов россия гост говядина антрекот бк охла россия коробка склад от кг до кг'

#### Разделлим данные на обучающую, тестовую и валидационную выборку

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'].values.astype('U'), df['mtype'], test_size=0.2, random_state=12345)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train, random_state=12345)

display(y_train.value_counts())
display(y_test.value_counts())
display(y_val.value_counts())

1    6114
4    2190
3    1103
2     949
0     805
5     681
Name: mtype, dtype: int64

1    1630
4     621
3     345
2     285
0     222
5     187
Name: mtype, dtype: int64

1    680
4    243
3    123
2    105
0     89
5     76
Name: mtype, dtype: int64

In [ ]:
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words = stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
x_train_tf = count_tf_idf.fit_transform(x_train)

x_val_tf = count_tf_idf.transform(x_val)
x_test_tf = count_tf_idf.transform(x_test)

#### Перевод матрицы tf в тензор

In [ ]:
x_train_tf = torch.tensor(scipy.sparse.csr_matrix.todense(x_train_tf)).float()
x_val_tf = torch.tensor(scipy.sparse.csr_matrix.todense(x_val_tf)).float()
x_test_tf = torch.tensor(scipy.sparse.csr_matrix.todense(x_test_tf)).float()

x_train_tf.shape

torch.Size([11842, 6161])

#### Конвертируем целевой признак в тензор

In [ ]:
y_train.values

array([2, 4, 1, ..., 0, 1, 1])

In [ ]:
y_train = torch.tensor(y_train.values).long()
y_val = torch.tensor(y_val.values).long()
y_test = torch.tensor(y_test.values).long()

In [ ]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
train_dataset = ClassifierDataset(x_train_tf, y_train)
val_dataset = ClassifierDataset(x_val_tf, y_val)
test_dataset = ClassifierDataset(x_test_tf, y_test)

In [ ]:
EPOCHS = 15
BATCH_SIZE = 16

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

#### Создаём модель

In [ ]:
SEED = 12345
torch.manual_seed(SEED)

In [ ]:
class MulticlassClassification(nn.Module):
  def __init__(self, num_feature, num_class = 6):
      super(MulticlassClassification, self).__init__()
      self.layer_1 = nn.Linear(num_feature, 512)
      self.layer_2 = nn.Linear(512, 256)
      self.layer_3 = nn.Linear(256, 128)
      self.layer_out = nn.Linear(128, num_class)

      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(p=0.2)
      self.batchnorm1 = nn.BatchNorm1d(512)
      self.batchnorm2 = nn.BatchNorm1d(256)
      self.batchnorm3 = nn.BatchNorm1d(128)
      
  def forward(self, x):
     x = self.layer_1(x)
     x = self.batchnorm1(x)
     x = self.relu(x)   

     x = self.layer_2(x)
     x = self.batchnorm2(x)
     x = self.relu(x)
     x = self.dropout(x)

     x = self.layer_3(x)
     x = self.batchnorm3(x)
     x = self.relu(x)
     x = self.dropout(x)
        
     x = self.layer_out(x)
        
     return x

In [ ]:
def multi_acc(y_pred, y_test):
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
  correct_pred = (y_pred_tags == y_test).long()
  acc = correct_pred.sum() / len(correct_pred)
  acc = torch.round(acc) * 100
  return acc

In [ ]:
accuracy_stats = {'train': [],"val": []}
loss_stats = {'train': [],"val": []}

In [ ]:
model = MulticlassClassification(num_feature = x_train_tf.shape[1], num_class=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

MulticlassClassification(
  (layer_1): Linear(in_features=6161, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=6, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.is_available()
#model.to(device)

cpu


False

In [ ]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
  train_epoch_loss = 0
  train_epoch_acc = 0
  model.train()
  count = 0

  for X_train_batch, y_train_batch in train_loader:
     optimizer.zero_grad()
     y_train_pred = model(X_train_batch)
     train_loss = criterion(y_train_pred, y_train_batch)
     train_acc = multi_acc(y_train_pred, y_train_batch)
     train_loss.backward()
     optimizer.step()
     train_epoch_loss += train_loss.item()
     train_epoch_acc += train_acc.item()
     count+=1

  print(count)
  train_epoch_acc /= 100*count
  train_epoch_loss /= 100*count

  with torch.no_grad():
      val_epoch_loss = 0
      val_epoch_acc = 0
      model.eval()
      count = 0
      for X_val_batch, y_val_batch in val_loader:
        
        y_val_pred = model(X_val_batch)
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = multi_acc(y_val_pred, y_val_batch)
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()
        count+=1
      print(count)
      val_epoch_acc /= 100*count
      val_epoch_loss /= 100*count

  print(f"Epoch: {e}/{EPOCHS}.. ",
          f"Training Loss: {train_epoch_loss:.3f}.. ",
          f"Training Accuracy: {train_epoch_acc:.3f}.. ",
          f"Val Loss: {val_epoch_loss:.3f}.. ",
          f"Val Accuracy: {val_epoch_acc:.3f}")

loss_stats['train'].append(train_epoch_loss/len(train_loader))
accuracy_stats['train'].append(train_epoch_acc/len(train_loader))



Begin training.


  0%|          | 0/15 [00:00<?, ?it/s]

741
1316
Epoch: 1/15..  Training Loss: 0.003..  Training Accuracy: 0.985..  Val Loss: 0.002..  Val Accuracy: 0.936
741
1316
Epoch: 2/15..  Training Loss: 0.001..  Training Accuracy: 0.999..  Val Loss: 0.002..  Val Accuracy: 0.947
741
1316
Epoch: 3/15..  Training Loss: 0.001..  Training Accuracy: 0.999..  Val Loss: 0.002..  Val Accuracy: 0.948
741
1316
Epoch: 4/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val Loss: 0.002..  Val Accuracy: 0.946
741
1316
Epoch: 5/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val Loss: 0.003..  Val Accuracy: 0.945
741
1316
Epoch: 6/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val Loss: 0.003..  Val Accuracy: 0.947
741
1316
Epoch: 7/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val Loss: 0.003..  Val Accuracy: 0.943
741
1316
Epoch: 8/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val Loss: 0.002..  Val Accuracy: 0.949
741
1316
Epoch: 9/15..  Training Loss: 0.000..  Training Accuracy: 0.999..  Val 

In [ ]:
y_pred_list = []
with torch.no_grad():
   model.eval()
   for X_batch, _ in test_loader:
     y_test_pred = model(X_batch)
     y_pred_softmax = torch.log_softmax(y_test_pred, dim = 1)
     _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
     y_pred_list.append(y_pred_tags.cpu().numpy())

y_pred_list[:10]

[array([2]),
 array([3]),
 array([1]),
 array([3]),
 array([1]),
 array([5]),
 array([1]),
 array([0]),
 array([1]),
 array([1])]

In [ ]:
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list[:10]

[2, 3, 1, 3, 1, 5, 1, 0, 1, 1]

In [ ]:
print("classification report :\n",classification_report(y_test, y_pred_list), sep='')

classification report :
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       222
           1       0.99      0.98      0.99      1630
           2       0.99      1.00      0.99       285
           3       0.78      0.78      0.78       345
           4       0.98      0.99      0.98       621
           5       0.61      0.67      0.64       187

    accuracy                           0.94      3290
   macro avg       0.89      0.90      0.89      3290
weighted avg       0.95      0.94      0.95      3290



## Вывод
Из-за дисбаланса классов, модель плохо распознаёт классы 3, 5. Это виднопо значеням метрики precision, recall, f1-score. Но метрика F1weight = 0.94, что в целом является неплохим результатом.

## Сделаем проверку обученной модели

In [ ]:
data_test = {'text': ['Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.',
                      'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:',
                      'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.'], 'target': [-1, -1, -1]}
data_test = pd.DataFrame(data = data_test)
data_test           

,text,target
0,"Говядина блочная 2 сорт в наличии ООО ""АгроСою...",-1
1,Куриная разделка Продам кур и куриную разделку...,-1
2,Говяжью мукозу Продам говяжью мукозу в охл и з...,-1


In [ ]:
data_test['text'] = data_test['text'].apply(lambda x : ''.join(m.lemmatize(x)))
data_test 

,text,target
0,"говядина блочный 2 сорт в наличие ооо ""агросою...",-1
1,куриный разделка продавать курица и куриный ра...,-1
2,говяжий мукоза продавать говяжий мукоза в охла...,-1


In [ ]:
data_test['text'] = data_test['text'].apply(lambda x : ' '.join(re.sub(r'[^а-яА-ЯёЁ]', ' ', x.lower()).split()))
data_test

,text,target
0,говядина блочный сорт в наличие ооо агросоюз р...,-1
1,куриный разделка продавать курица и куриный ра...,-1
2,говяжий мукоза продавать говяжий мукоза в охла...,-1


In [ ]:
data_test_x = count_tf_idf.transform(data_test['text'])

data_test_x = torch.tensor(scipy.sparse.csr_matrix.todense(data_test_x)).float()

data_test_y = torch.tensor(data_test['target']).long()

testing = ClassifierDataset(data_test_x, data_test_y)

testing = DataLoader(dataset=testing, batch_size=1)


In [ ]:
y_pred_list = []
with torch.no_grad():
   model.eval()
   for X_batch,_ in testing:
     y_test_pred = model(X_batch)
     y_pred_softmax = torch.log_softmax(y_test_pred, dim = 1)
     _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
     y_pred_list.append(y_pred_tags.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
list(le.inverse_transform(y_pred_list))

['говядина', 'кура', 'говядина']

## Вывод
Рузультат на тесте показал, что модель довольно точно определяет классы у 3 тестовых текстов